## XGBoost Model deployment in Amazon Sagemaker. 

#### This notebook should be run in an Amazon Sagemaker notebook instance. 


#### Before running this notebook, 
you should have uploaded the pre-trained model and test_point.csv from your laptop to the 
same folder where you have this notebook file. test_point.csv contains few sample test data in csv format.


This loads the pre-trained XGBoost model and saves in a S3 bucket in .tar.gz format as required by Sagemaker.
Then it creates a sagemaker model from the model file stored in S3. 
Then configures and creates an Endpoint to deploy the model and also tests invoking the endpoint to get prediction.

#### Please remember not to run the last "Delete the Endpoint" cell if you want to test the deployed model from a client. 


After the exercise is over, 
##### you should cleanup the Sagemaker resources as described in 
https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html to avoid charges incurred because of resources left behind.




### Import libraries

In [1]:
%%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

CPU times: user 792 ms, sys: 196 ms, total: 988 ms
Wall time: 8.41 s


### Create S3 bucket

In [2]:
# This creates a default S3 bucket where we will upload our model.
bucket = sagemaker.Session().default_bucket()

In [3]:
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [4]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::018663355535:role/service-role/AmazonSageMaker-ExecutionRole-20210521T154643
ap-south-1
sagemaker-ap-south-1-018663355535
https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-018663355535


#### Install xgboost as it is needed for loading the model from joblib dump file and test it before deployment.
#### Please note that the XGBoost version should be same as the version with which the model was trained locally in laptop.

In [5]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::nbclassic==0.2.6=pyhd8ed1ab_0
  - conda-forge/linux-64::blaze==0.11.3=py36_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::jupyterlab==3.0.9=pyhd8ed1ab_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/linux-64::bo

In [6]:
model_file_name = "DEMO-local-xgboost-model"

### Load the pre-trained model and test it before deployment

In [7]:
import joblib
import xgboost

mymodel = joblib.load(model_file_name)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
#import json
import numpy as np


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)    

[[5.4 3.  4.5 1.5]
 [5.6 3.  4.1 1.3]
 [6.3 2.8 5.1 1.5]
 [6.  3.  4.8 1.8]
 [5.1 3.3 1.7 0.5]]


In [9]:
mymodel.predict(mypayload)

array([1, 1, 1, 2, 0], dtype=int32)

#### Create a tar.gz model file as this is the format required by Sagemaker for deployment.

In [10]:
#### This step Booster.save_model was needed before creating a tar.gz . Otherwise I faced issues with prediction on deployment.

mymodel._Booster.save_model(model_file_name)

In [11]:
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost-model


### Upload the pre-trained model to S3

In [12]:
#### prefix in S3
prefix = "sagemaker/DEMO-xgboost-byo"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/DEMO-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz


### Set up hosting for the model¶
#### Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.

#### Create a Sagemaker model 

In [13]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [14]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-018663355535/sagemaker/DEMO-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz
arn:aws:sagemaker:ap-south-1:018663355535:model/demo-local-xgboost-model2021-05-26-01-42-10
CPU times: user 59.4 ms, sys: 17.1 ms, total: 76.5 ms
Wall time: 438 ms


### Create endpoint configuration

Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [15]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoostEndpointConfig-2021-05-26-01-42-12
Endpoint Config Arn: arn:aws:sagemaker:ap-south-1:018663355535:endpoint-config/demo-xgboostendpointconfig-2021-05-26-01-42-12


### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [16]:
%%time
import time

endpoint_name = "DEMO-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoostEndpoint-2021-05-26-01-42-15
arn:aws:sagemaker:ap-south-1:018663355535:endpoint/demo-xgboostendpoint-2021-05-26-01-42-15
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:ap-south-1:018663355535:endpoint/demo-xgboostendpoint-2021-05-26-01-42-15
Status: InService
CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 7min


### Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

In [17]:
runtime_client = boto3.client("runtime.sagemaker")

Lets generate the prediction. We'll pick csv data from the test data file

In [18]:
%%time
import json


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# Unpack response
print("\nPredicted Class Probabilities: {}.".format(result))

Payload :

5.400000000000000355e+00,3.000000000000000000e+00,4.500000000000000000e+00,1.500000000000000000e+00
5.599999999999999645e+00,3.000000000000000000e+00,4.099999999999999645e+00,1.300000000000000044e+00
6.299999999999999822e+00,2.799999999999999822e+00,5.099999999999999645e+00,1.500000000000000000e+00
6.000000000000000000e+00,3.000000000000000000e+00,4.799999999999999822e+00,1.800000000000000044e+00
5.099999999999999645e+00,3.299999999999999822e+00,1.699999999999999956e+00,5.000000000000000000e-01

Results :



Predicted Class Probabilities: [0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.08053058385848999, 0.8385403752326965, 0.08092901110649109],[0.12150664627552032, 0.27632802724838257, 0.6021653413772583],[0.8827555775642395, 0.0601295568048954, 0.05711490288376808].
CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 133 ms


### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [19]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'ebcf9cce-8c90-4086-b593-0fc7056e4685',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ebcf9cce-8c90-4086-b593-0fc7056e4685',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 26 May 2021 01:50:00 GMT'},
  'RetryAttempts': 0}}